In [1]:
!pip install sentence-transformers
!pip install transformers
!pip install spacy
!pip install temporal_taggers
!pip install gensim
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16164 sha256=848e6a4ea14c1370914b903ca2b873ad8429f0810af2329c2de4d7eb2e57a7fb
  Stored in directory: /root/.cache/pip/wheels/9c/d6/00/1ccfd5a7466a94774e00022683d4b028836032dfb85007822b
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 44.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 46.4 MB/s eta 0:00:0000:01


In [2]:
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 72.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
import pickle

def save_embeddings_to_file(embedding_index, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(embedding_index, f)

def load_embeddings_from_file(file_path):
    with open(file_path, 'rb') as f:
        embedding_index = pickle.load(f)
    return embedding_index

embedding_file = '/notebooks/glove.6B.300d.txt'
pickle_file_path = '/notebooks/embedding_index.pkl'

# Load the embeddings from the text file and create the embedding_index
embedding_index = {}
with open(embedding_file, encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = vector

# Save the embedding_index to a pickle file
save_embeddings_to_file(embedding_index, pickle_file_path)


In [3]:
import pandas as pd
import string
import re
import pickle
import spacy
from transformers import AutoTokenizer, BertForTokenClassification
import torch
from temporal_taggers.evaluation import merge_tokens
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
from itertools import chain
from concurrent.futures import ThreadPoolExecutor
from Levenshtein import distance as levenshtein_distance
from sklearn.preprocessing import StandardScaler, LabelEncoder
import xgboost as xgb
import time
from functools import partial

model = BertForTokenClassification.from_pretrained("satyaalmasian/temporal_tagger_BERT_tokenclassifier")
text_tokenizer = AutoTokenizer.from_pretrained("satyaalmasian/temporal_tagger_BERT_tokenclassifier", use_fast=False)
sentence_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
# Load the classifier pipeline
classifier = pipeline("text-classification", model="microsoft/deberta-v2-xlarge-mnli", top_k=None, framework="pt", device=0, batch_size=100)
glove_model = KeyedVectors.load_word2vec_format('glove.6B.300d.word2vec.bin', binary=True)

# Load embeddings
def load_embeddings_from_file(file_path):
    with open(file_path, 'rb') as f:
        embedding_index = pickle.load(f)
    return embedding_index

pickle_file_path = '/notebooks/embedding_index.pkl'

# Load the embeddings from the text file and create the embedding_index
embedding_index = load_embeddings_from_file(pickle_file_path)

            
def predict(df):
    # Load json and convert to dataframe foramt
    # Do Basic Cleaning, also drop any empty or NA rows
    # We do not remove duplicates, since there might be use cases that require duplicate predictions (but the label will be the same..)
    start_time = time.time()
    df = drop_empty_rows(df, "claim", "evidence")

    ## Apply preprocessing function to claim and evidence_content columns of dataframe
    df['claim'] = df['claim'].apply(preprocess_string)
    df['evidence'] = df['evidence'].apply(preprocess_string)

    nlp = spacy.load("en_core_web_sm")

    # Use nlp.pipe for efficient processing of multiple texts
    claims = list(df['claim'])
    evidences = list(df['evidence'])

    # Set batch_size and n_process for faster processing
    batch_size = 500
    n_process = -1  # Use all available cores

    # Process claims and evidences using nlp.pipe with batch_size and n_process
    claims_docs = list(nlp.pipe(claims, batch_size=batch_size, n_process=n_process))
    evidences_docs = list(nlp.pipe(evidences, batch_size=batch_size, n_process=n_process))

    # Extract entities for all docs
    claim_named_entities_list, claim_entities, claim_chunks, claim_temporal = extract_entities_and_chunks(claims_docs)
    evidence_named_entities_list, evidence_entities, evidence_chunks, evidence_temporal = extract_entities_and_chunks(evidences_docs)

    df['claim_named_entities'] = claim_named_entities_list
    df['claim_entities'] = claim_entities
    df['claim_chunks'] = claim_chunks
    df['evidences_named_entities'] = evidence_named_entities_list
    df['evidence_entities'] = evidence_entities
    df['evidence_chunks'] = evidence_chunks

    df['claim_temporal'] = claim_temporal
    df['evidence_temporal'] = evidence_temporal

    # Set batch size and number of batches
    batch_size = 500
    num_batches = len(df) // batch_size + 1

    # Initialize an empty list to store the similarity scores
    similarity_scores = []

    # Compute the similarity score for each row in the dataframe using batching
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(df))
        batch_df = df.iloc[start_idx:end_idx]

        claims = list(batch_df['claim'])
        evidences = list(batch_df['evidence'])

        # Compute the similarity scores for the batch using the `compute_similarity()` function
        batch_scores = []
        for j, claim in enumerate(claims):
            doc_scores = compute_similarity(claim, evidences[j])
            batch_scores.append(doc_scores)

        similarity_scores += batch_scores

    # Flatten the similarity scores list
    similarity_scores = [score for batch_scores in similarity_scores for score in batch_scores]

    # Add the similarity score as a new column to the dataframe
    df['similarity_score'] = similarity_scores
    
    ## Add Deberta V2
    # Split the data into batches and process each batch
    batch_size = 100
    dfs = []
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i+batch_size]
        result = add_probability_columns(batch)
        dfs.append(result)

    # Concatenate the results
    df = pd.concat(dfs)

    ## Add counts of named entities only
    df['claim_named_entities'] = df['claim_named_entities'].astype(str).apply(eval)
    df['evidences_named_entities'] = df['evidences_named_entities'].astype(str).apply(eval)

    df['count_claim_named_entities'] = df['claim_named_entities'].apply(count_list_items)
    df['count_evidence_named_entities'] = df['evidences_named_entities'].apply(count_list_items)
    
    # Take the probabilities out of the brackets
    df['contradiction_prob'] = df['contradiction_prob'].astype(str).apply(eval)
    df['neutral_prob'] = df['neutral_prob'].astype(str).apply(eval)
    df['entailment_prob'] = df['entailment_prob'].astype(str).apply(eval)

    # apply function to DataFrame column
    df['contradiction_prob'] = df['contradiction_prob'].apply(extract_float)
    df['neutral_prob'] = df['neutral_prob'].apply(extract_float)
    df['entailment_prob'] = df['entailment_prob'].apply(extract_float)
    
    # Add similarity metrics between entities, chunks, and temporal phrases, load glove from binary file

    # Prepare data for parallel processing
    data = range(len(df))

    # Run the function in parallel using a ThreadPoolExecutor
    partial_row_similarity = partial(row_similarity, df)
    with ThreadPoolExecutor() as executor:
        similarity_scores = list(executor.map(partial_row_similarity, data))

    # Unpack the chunk_similarity and entity_similarity scores
    chunk_similarity, entity_similarity, named_entity_similarity, temporal_similarity = zip(*similarity_scores)

    # Add similarity scores to the DataFrame
    df['chunks_similarity'] = chunk_similarity
    df['entities_similarity'] = entity_similarity
    df['named_entity_similarity'] = named_entity_similarity
    df['temporal_similarity'] = temporal_similarity
    
    # Add embedding
    # Apply the compute_average_embedding function to the claim and evidence columns to obtain the average word embeddings
    df['claim_embedding'] = df['claim'].apply(compute_average_embedding)
    df['evidence_embedding'] = df['evidence'].apply(compute_average_embedding)

    processed_df = df.iloc[:, 10:]
    processed_df = processed_df.rename(columns={'chunks_similarity': 'all_chunks_similarity', 'named_entity_similarity': 'named_entities_similarity', 'entities_similarity': 'all_entities_similarity'})
    processed_df# reorder columns
    new_order = ['contradiction_prob', 'neutral_prob', 'entailment_prob','count_claim_named_entities', 'count_evidence_named_entities',
           'similarity_score', 'all_entities_similarity',
           'all_chunks_similarity', 'named_entities_similarity',
           'temporal_similarity', 'claim_embedding', "evidence_embedding"] 
    processed_df = processed_df.reindex(columns=new_order)

    # Standardize the numerical features
    scaler = StandardScaler()
    processed_df_scaled = scaler.fit_transform(processed_df.iloc[:, :10])
    # Combine the standardized numerical features with the claim and evidence embeddings
    claim_embeddings = np.vstack(processed_df['claim_embedding'].values)
    evidence_embeddings = np.vstack(processed_df['evidence_embedding'].values)
    processed_df_scaled_emb = np.hstack((processed_df_scaled, claim_embeddings, evidence_embeddings))

    # Load the model from file
    model = xgb.XGBClassifier()
    model.load_model("xgboost_final_reduced.model")
    # Make predictions on new data
    predictions = model.predict(processed_df_scaled_emb)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time:.4f} seconds")
    return predictions

def drop_empty_rows(df, col1, col2):
    # Drop rows with NaN or null values in col1 and col2
    df = df.dropna(subset=[col1, col2], how='any')

    # Drop rows with empty strings in col1 and col2
    df = df[~df[col1].str.strip().eq('')]
    df = df[~df[col2].str.strip().eq('')]

    return df.reset_index(drop=True)

def preprocess_string(text):
    # Remove unnecessary whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove non-alphanumeric characters except for whitespace and normal punctuation
    text = re.sub(r'[^\w\s\'",.?!:;\-]|[\x80-\xff]', '', text)

    # Remove consecutive punctuation
    text = re.sub(r'([,.?!:;\-])\1+', r'\1', text)

    # Remove leading and trailing punctuation except for normal end-of-sentence punctuation
    text = re.sub(r'^(\'|"|\(|\[|\{)*\s*(.*?)[\s\'",:;!?]*(\.|\?|\!)(\'|"|\)|\]|\})*\s*$', r'\2\3', text)

    # Remove spaces before commas and full stops
    text = re.sub(r'\s+([,.])', r'\1', text)

    # Fix possessive apostrophe
    text = re.sub(r'\s+\'s\b', '\'s', text)

    # Remove unnecessary quotes and spaces
    text = re.sub(r'\'\'|\s*"\s*|\s*\'', ' ', text).strip()

    # Remove all remaining extra spaces
    text = re.sub(r'\s+', ' ', text)

    return text

# Define a function to extract all entities, and noun chunks
def extract_entities_and_chunks(docs):
    named_entities_list = []
    entities_list = []
    chunks_list = []
    temporal_list = []
    for doc in docs:
        # Extract entities and noun chunks
        entities = [ent.text for ent in doc.ents if ent.label_ not in ["DATE", "TIME"]]
        entities_list.append(entities)
        noun_chunks = [chunk.text for chunk in doc.noun_chunks if not any(ent.label_ in ["DATE", "TIME"] for ent in chunk.ents)]
        chunks_list.append(noun_chunks)

        # Extract named entities 
        named_entities = [ent.text for ent in doc.ents]
        named_entities_list.append(named_entities)

        # Extract temporal phrases
        text = doc.text
        temporal_phrases = extract_temporal_phrases(text)
        temporal_list.append(temporal_phrases)


    # Return the named entities, entities and noun chunks and temporal phrases 
    return named_entities_list, entities_list, chunks_list, temporal_list

# Define a function to extract temporal elements
def extract_temporal_phrases(text):
    id2label = {v: k for k, v in model.config.label2id.items()}
    text = text.lower()
    processed_text = text_tokenizer(text, return_tensors="pt")
    result = model(**processed_text)
    classification = torch.argmax(result[0], dim=2)
    merged_tokens = merge_tokens(processed_text["input_ids"][0], classification[0], id2label, text_tokenizer)
    temporal_phrases = []
    temporal = False
    for token in merged_tokens:
      if token[1] == "DATE" or token[1] == "TIME" or token[1] == "DURATION":
          if temporal:
              temporal_phrases[-1] += " " + token[0]
          else:
              temporal_phrases.append(token[0])
              temporal = True
      else:
          temporal = False

    temporal_phrases = [str(re.sub(r'[^\w\s]', '', phrase.strip())) for phrase in temporal_phrases]
    temporal_phrases = list(set([re.sub(r'\s+', ' ', phrase.strip()) for phrase in temporal_phrases]))

    return temporal_phrases

# Define a function to compute the similarity score between a query and a document
def compute_similarity(query, doc):
    # Encode the query and document
    query_emb = sentence_model.encode(query)
    doc_emb = sentence_model.encode(doc)

    #Compute dot score between query and all document embeddings
    scores = util.dot_score(query_emb, doc_emb)[0].tolist()

    return scores

def add_probability_columns(df):
    # Define input pairs from DataFrame
    input_pairs = list(zip(df['claim'].tolist(), df['evidence'].tolist()))

    # Combine input pairs into single strings with [CLS] and [SEP] tokens
    inputs = [f"[CLS]{pair[0]} [SEP] {pair[1]} [SEP]" for pair in input_pairs]

    # Run classification on batch
    results = classifier(inputs, max_length=128)

    # Initialize lists for each label's probability
    contradiction_prob = [[] for _ in range(len(df))]
    neutral_prob = [[] for _ in range(len(df))]
    entailment_prob = [[] for _ in range(len(df))]

    # Extract probability scores for each input pair
    for i, result in enumerate(results):
        for score in result:
            if score['label'] == 'CONTRADICTION':
                contradiction_prob[i].append(score['score'])
            elif score['label'] == 'NEUTRAL':
                neutral_prob[i].append(score['score'])
            elif score['label'] == 'ENTAILMENT':
                entailment_prob[i].append(score['score'])

    # Add probability columns to original DataFrame
    df['contradiction_prob'] = contradiction_prob
    df['neutral_prob'] = neutral_prob
    df['entailment_prob'] = entailment_prob

    return df

def count_list_items(s):
    return (len(s))

def extract_float(lst):
    return lst[0]

def normalized_levenshtein_distance(a, b):
    max_length = max(len(a), len(b))
    if max_length == 0:
        return 0
    return levenshtein_distance(a, b) / max_length

def preprocess_keyword(keyword):
    keyword = re.sub(r'[^A-Za-z0-9 ]+', '', keyword)  # Remove non-alphanumeric characters except space
    keyword = re.sub(' +', ' ', keyword)
    keyword = keyword.strip()  # Remove leading and trailing spaces
    return keyword.split(" ")  # Return the preprocessed keyword

def avg_similarity(claim_keywords, evidence_keywords, model, alpha=0.5):
    total_similarity = 0
    count = 0

   # Preprocess claim keywords and evidence keywords
    claim_keywords = [word for keyword in claim_keywords for word in preprocess_keyword(keyword)]
    evidence_keywords = [word for keyword in evidence_keywords for word in preprocess_keyword(keyword)]

    for claim_keyword in claim_keywords:
        max_similarity = -1

        for evidence_keyword in evidence_keywords:
            # Check if words are in the model vocabulary
            claim_in_vocab = claim_keyword in model
            evidence_in_vocab = evidence_keyword in model

            embedding_similarity = 0

            if claim_in_vocab and evidence_in_vocab:
              # Calculate the cosine similarity using word embeddings
              embedding_similarity = cosine_similarity([model[claim_keyword]], [model[evidence_keyword]])

            # Calculate the normalized Levenshtein distance
            levenshtein_sim = 1 - normalized_levenshtein_distance(claim_keyword, evidence_keyword)

            # Combine the embedding similarity and Levenshtein similarity with a weighted average
            combined_similarity = alpha * embedding_similarity + (1 - alpha) * levenshtein_sim

            max_similarity = max(max_similarity, combined_similarity)

        total_similarity += max_similarity
        count += 1

    return total_similarity / count if count > 0 else 0

# Function to apply the avg_similarity on DataFrame rows
def row_similarity(df, index):
    row = df.loc[index]
    claim_chunks = row['claim_chunks']
    evidence_chunks = row['evidence_chunks']
    chunk_similarity = avg_similarity(claim_chunks, evidence_chunks, glove_model, alpha=0.5)

    claim_entities = row['claim_entities']
    evidence_entities = row['evidence_entities']
    entity_similarity = avg_similarity(claim_entities, evidence_entities, glove_model, alpha=0.5)

    claim_named_entities = row['claim_named_entities']
    evidence_named_entities = row['evidences_named_entities']
    named_entity_similarity = avg_similarity(claim_named_entities, evidence_named_entities, glove_model, alpha=0.5)

    claim_temporal = row['claim_temporal']
    evidence_temporal = row['evidence_temporal']
    temporal_similarity = avg_similarity(claim_temporal, evidence_temporal, glove_model, alpha=0.5)

    return chunk_similarity, entity_similarity, named_entity_similarity, temporal_similarity

# Define a function to compute the average word embedding for a sentence
def compute_average_embedding(sentence):
    words = sentence.split()
    embeddings = []
    for word in words:
        if word in embedding_index:
            embeddings.append(embedding_index[word])
    if len(embeddings) > 0:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(embedding_dim)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Load Data
# We assume here that data is passed in the correct format
import pandas as pd
df = pd.read_csv("/notebooks/trial_preprocessing.csv", encoding='utf-8')
predict(df)